## Load Data Set

In [1]:
import zipfile
import os
import numpy as np
import pathlib
import pandas as pd
from math import ceil
import tensorflow as tf
import numpy as np
import IPython.display as display
import keras
from keras import backend as K
from matplotlib import pyplot as plt
from keras.utils import to_categorical


print(tf.__version__)

root_path = './'
dataset_path = os.path.join(root_path, 'tf_dataset_gray')

models_path = os.path.join(root_path, 'saved_models_gray')
if not os.path.exists(models_path):
  os.mkdir(models_path)

Using TensorFlow backend.


1.13.1


In [2]:
BATCH_SIZE = 32


image_feature_description = {
    'enc': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'age': tf.io.FixedLenSequenceFeature([], tf.int64, allow_missing=True),
    'img': tf.io.FixedLenSequenceFeature([], tf.string, allow_missing=True)
}


def _parse_image_function(example_proto):
    return tf.io.parse_single_example(example_proto, image_feature_description)


raw_train_dataset = tf.data.TFRecordDataset(os.path.join(dataset_path,'train.tfrecords'))
parsed_train_dataset = raw_train_dataset.map(_parse_image_function)

# raw_val_dataset = tf.data.TFRecordDataset(os.path.join(dataset_path, 'val.tfrecords'))
# parsed_val_dataset = raw_val_dataset.map(_parse_image_function)

# raw_test_dataset = tf.data.TFRecordDataset(os.path.join(dataset_path, 'test.tfrecords'))
# parsed_test_dataset = raw_test_dataset.map(_parse_image_function)


parsed_train_dataset = parsed_train_dataset.repeat()
parsed_train_dataset = parsed_train_dataset.shuffle(6000)
parsed_train_dataset = parsed_train_dataset.batch(BATCH_SIZE)
dataset_iterator = parsed_train_dataset.make_one_shot_iterator()

variable_dataset = dataset_iterator.get_next()

## Wasserstein GAN

In [3]:
def d_loss(y_true, y_pred):
    return K.mean(y_true * y_pred)

## Model Design

In [4]:
enc_len = 0
age_len = 0
img_shape = (32, 32, 1)
width, height, depth = (32, 32, 1)
img_len = np.prod(img_shape)
latent_dim = enc_len + age_len + img_len
noise_len = 100
input_dim = enc_len + age_len + noise_len
cond_len = enc_len + age_len
ngf = 64
ndf = 32


def build_discriminator():
    conv = keras.Sequential([
        # conv block 1
        keras.layers.Conv2D(
            filters=ndf,
            kernel_size=(4, 4),
            strides=2,
            input_shape=img_shape,
            use_bias=False
        ),
        keras.layers.Activation(tf.nn.leaky_relu),
        
        # conv block 2
        keras.layers.Conv2D(
            filters=ndf * 2,
            kernel_size=(4, 4),
            strides=2,
            use_bias=False
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.leaky_relu),
        
        # conv block 3
        keras.layers.Conv2D(
            filters=ndf * 4,
            kernel_size=(4, 4),
            strides=2,
            use_bias=False
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.leaky_relu),
        
        # conv block 5
        keras.layers.Conv2D(
            filters=1,
            kernel_size=(1, 1),
            strides=2,
            use_bias=False
        )
    ])
    
    conv.summary()
    
    model = keras.Sequential([        
        # output
        keras.layers.Activation(tf.nn.sigmoid, input_shape=(1,)),
    ])
    
    model.summary()
    
    # image
    z = keras.layers.Input(shape=img_shape)
    
    # convolution
    zout = conv(z)
    
    # flatten image
    z_flat = keras.layers.Flatten()(zout)
    
    # concatenation
#     inputs = keras.layers.concatenate([z_flat])
    
    # real or fake
    outputs = model(z_flat)
    
    # age label
#     classes = clf(inputs)
    
    
    return keras.models.Model(z, outputs)


def build_generator():
    
    conv = keras.Sequential([
        # transpose conv block 1
        keras.layers.Conv2DTranspose(
            filters=ngf * 8,
            kernel_size=(4, 4),
            strides=2,
            input_shape=(10, 10, 1),
            use_bias=False
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 2
        keras.layers.Conv2DTranspose(
            filters=ngf * 4,
            kernel_size=(4, 4),
            strides=1,
            use_bias=False
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 3
        keras.layers.Conv2DTranspose(
            filters=ngf * 2,
            kernel_size=(4, 4),
            strides=1,
            use_bias=False
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 4
        keras.layers.Conv2DTranspose(
            filters=ngf * 2,
            kernel_size=(4, 4),
            strides=1,
            use_bias=False
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 5
        keras.layers.Conv2DTranspose(
            filters=1,
            kernel_size=(2, 2),
            strides=1,
            use_bias=False
        ),
        
        # output
        keras.layers.Activation(tf.nn.tanh)
    ])
    
    conv.summary()
    
    model = keras.Sequential([        
        # reshape 1d to 3d
        keras.layers.Reshape((10, 10, 1), input_shape=(input_dim,))
    ])
    
    model.summary()
    
    # noise
    x = keras.layers.Input(shape=(noise_len,))
    
    # flat dense output
    out_1 = model(x)
    
    # transpose conv output
    outputs = conv(out_1)
    
    return keras.models.Model(x, outputs)


discriminator = build_discriminator()
generator = build_generator()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 15, 15, 32)        512       
_________________________________________________________________
activation_1 (Activation)    (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 64)          32768     
_________________________________________________________________
batch_normalization_1 (Batch (None, 6, 6, 64)          256       
_________________________________________________________________
activation_2 (Activation)    (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 2, 2, 128)         131072    
_________________________________________________________________
batc

In [ ]:
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
sequential_4 (Sequential)    (None, 10, 10, 1)         0         
_________________________________________________________________
sequential_3 (Sequential)    (None, 32, 32, 1)         2896384   
Total params: 2,896,384
Trainable params: 2,894,336
Non-trainable params: 2,048
_________________________________________________________________


In [ ]:
discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 1)         0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 1, 1, 1)           165248    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1)                 0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 1)                 0         
Total params: 165,248
Trainable params: 164,864
Non-trainable params: 384
_________________________________________________________________


## Generative Adversarial Network

In [ ]:
GLR = 0.0002  # generator
DLR = 0.0002  # discriminator


discriminator.compile(
    optimizer=keras.optimizers.Adam(DLR, 0.5),
    loss=keras.losses.binary_crossentropy,
    metrics=['accuracy']
)

# noise
x = keras.layers.Input(shape=(noise_len,))

# freeze discriminator
discriminator.trainable = False

# output
z = generator(x)
out = discriminator(z)

# GAN
gan = keras.models.Model(inputs=x, outputs=out)

gan.compile(
    optimizer=keras.optimizers.Adam(GLR , 0.5),
    loss=keras.losses.binary_crossentropy,
    metrics=['accuracy'])

In [ ]:
gan.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 100)               0         
_________________________________________________________________
model_2 (Model)              (None, 32, 32, 1)         2896384   
_________________________________________________________________
model_1 (Model)              (None, 1)                 165248    
Total params: 3,061,632
Trainable params: 2,894,336
Non-trainable params: 167,296
_________________________________________________________________


## Visualization Method

In [ ]:
# from google.colab import drive
import os


# drive.mount('/content/gdrive', force_remount=True)

root_path = './'
tgt_pth = os.path.join(root_path, 'visualize_age-cgan-v15')

if not os.path.exists(tgt_pth):
  os.mkdir(tgt_pth)

In [ ]:
def visualizeGAN(e, z_real, z_fake):

    fig, axes = plt.subplots(4, 4, figsize=(20, 18))

    r_real = 0
    r_fake = 0
    for row, axe in enumerate(axes):
        for col, cell in enumerate(axe):
            if row % 2 == 0:
                cell.imshow(
                    np.squeeze(
                        0.5 * z_real[r_real * 4 + col] + 0.5,
                        axis=-1
                    ),
                    cmap='gray'
                )
            else:
                cell.imshow(
                    np.squeeze(
                        0.5 * z_fake[r_fake * 4 + col] + 0.5,
                        axis=-1
                    ),
                    cmap='gray'
                )

            cell.axis("off")

        if row % 2 == 0:
            r_real += 1
        else:
            r_fake += 1

    plt.axis("off")
    plt.tight_layout()

    fig.savefig(os.path.join(tgt_pth, '{}.jpg'.format(str(e).zfill(3))))
    
    plt.close()

## Load Batch

In [ ]:
def load_noise():
    
    y_real = tf.ones((BATCH_SIZE,))
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allocator_type = 'BFC'
    config.gpu_options.per_process_gpu_memory_fraction = 0.40

    with tf.Session(config=config) as sess:
        tf.initialize_all_variables().run()
        
        # run once
        y_real = y_real.eval()

        while True:
            values = sess.run([variable_dataset])
            row = values[0]

            sz = row['img'].shape[0]

            if sz != BATCH_SIZE:
                continue
            
            # fake data
            # concatenate face + age + noise
#             c1 = row['enc']
#             c2 = tf.cast(row['age'], tf.float32).eval()
            x = tf.random.normal((sz, noise_len,)).eval()
            
            yield x, y_real


def load_batch():
    
    y_fake = tf.zeros((BATCH_SIZE,))
    y_real = tf.ones((BATCH_SIZE,))
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allocator_type = 'BFC'
    config.gpu_options.per_process_gpu_memory_fraction = 0.40

    with tf.Session(config=config) as sess:
        tf.initialize_all_variables().run()
        
        # run once
        y_fake = y_fake.eval()
        y_real = y_real.eval()

        while True:
            values = sess.run([variable_dataset])
            row = values[0]

            sz = row['img'].shape[0]

            if sz != BATCH_SIZE:
                continue
            
            # fake data
#             c1 = row['enc']
#             c2 = tf.cast(row['age'], tf.float32).eval()
            x = tf.random.normal((sz, noise_len,)).eval()
            z_fake = generator.predict(x)

            # real data
#             c1 = row['enc']
#             c2 = tf.cast(row['age'], tf.float32).eval()
            z_real = tf.reshape(tf.io.decode_raw(row['img'], tf.int64), (-1, width, height, depth))
    
            z_real = tf.cast(z_real, tf.float32)
    
            # scale to [-1, +1]
            z_real = tf.math.subtract(tf.math.divide(z_real, 127.5), 1)
        
            z_real = z_real.eval()
                        
            yield x, z_fake, y_fake, z_real, y_real

## Train Model

In [ ]:
EPOCHS = 4001
STEPS = 1  # 60000 // BATCH_SIZE


train_loss_g = []
train_loss_d = []

train_acc_g = []
train_acc_d = []


disc_itr = load_batch()
gen_itr = load_noise()

# to be visualized
next(disc_itr)
next(disc_itr)
next(disc_itr)
next(disc_itr)

_, _, _, viz_real, _ = next(disc_itr)
vis_noise = np.random.normal(size=(16, noise_len,))
print(vis_noise.shape)

# epochs
for e in range(EPOCHS):

    #batches
    loss = []
    acc = []

    for p in range(STEPS):
        
        x, z_fake, y_fake, z_real, y_real = next(disc_itr)

        # train
        loss_2, acc_2 = discriminator.train_on_batch(z_real, y_real)
        loss_1, acc_1 = discriminator.train_on_batch(z_fake, y_fake)

        batch_loss = 0.5 * (loss_1 + loss_2)
        batch_acc = 0.5 * (acc_1 + acc_2)

        loss.append(batch_loss)
        acc.append(batch_acc)

    train_loss_d.append(np.mean(np.array(loss)))
    train_acc_d.append(np.mean(np.array(acc)))

    #batches
    loss = []
    acc = []

    for p in range(STEPS):

        x, y_true = next(gen_itr)

        # train
        loss_1, acc_1 = gan.train_on_batch(x, y_true)

        loss.append(loss_1)
        acc.append(acc_1)

    train_loss_g.append(np.mean(np.array(loss)))
    train_acc_g.append(np.mean(np.array(acc)))


    print("E: {}, [D ACC: %{:.2f}], [D LOSS: {:.2f}], [G ACC: %{:.2f}], [G LOSS: {:.2f}]".format(
          e,
          train_acc_d[-1] * 100,
          train_loss_d[-1] * 100,
          train_acc_g[-1] * 100,
          train_loss_g[-1] * 100
      ))

    if e % 25 == 0:
        ## visualize results
        viz_fake = generator.predict(vis_noise)
        visualizeGAN(e, viz_real, viz_fake)
        
        ## save model
        pth = os.path.join(models_path, 'gan.h5')
        gan.save(pth)

        pth = os.path.join(models_path, 'generator-{}-{}-{}.h5'.format(e, train_loss_g[-1], train_acc_g[-1]))
        generator.save(pth)

        pth = os.path.join(models_path, 'discriminator.h5')
        discriminator.save(pth)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
(16, 100)
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


E: 0, [D ACC: %57.81], [D LOSS: 68.30], [G ACC: %81.25], [G LOSS: 50.54]
E: 1, [D ACC: %51.56], [D LOSS: 122.98], [G ACC: %93.75], [G LOSS: 24.87]
E: 2, [D ACC: %51.56], [D LOSS: 116.95], [G ACC: %84.38], [G LOSS: 39.97]
E: 3, [D ACC: %53.12], [D LOSS: 94.99], [G ACC: %81.25], [G LOSS: 39.81]
E: 4, [D ACC: %54.69], [D LOSS: 96.19], [G ACC: %81.25], [G LOSS: 48.04]
E: 5, [D ACC: %53.12], [D LOSS: 97.34], [G ACC: %90.62], [G LOSS: 43.49]
E: 6, [D ACC: %53.12], [D LOSS: 90.02], [G ACC: %84.38], [G LOSS: 47.00]
E: 7, [D ACC: %60.94], [D LOSS: 66.30], [G ACC: %90.62], [G LOSS: 38.16]
E: 8, [D ACC: %65.62], [D LOSS: 74.03], [G ACC: %93.75], [G LOSS: 33.47]
E: 9, [D ACC: %90.62], [D LOSS: 43.53], [G ACC: %84.38], [G LOSS: 39.15]
E: 10, [D ACC: %89.06], [D LOSS: 41.85], [G ACC: %62.50], [G LOSS: 70.37]
E: 11, [D ACC: %98.44], [D LOSS: 37.27], [G ACC: %28.12], [G LOSS: 99.55]
E: 12, [D ACC: %95.31], [D LOSS: 41.55], [G ACC: %21.88], [G LOSS: 112.82]
E: 13, [D ACC: %92.19], [D LOSS: 44.72], [G A

E: 111, [D ACC: %98.44], [D LOSS: 14.14], [G ACC: %100.00], [G LOSS: 4.14]
E: 112, [D ACC: %96.88], [D LOSS: 16.97], [G ACC: %100.00], [G LOSS: 3.13]
E: 113, [D ACC: %98.44], [D LOSS: 22.11], [G ACC: %100.00], [G LOSS: 4.48]
E: 114, [D ACC: %96.88], [D LOSS: 24.93], [G ACC: %100.00], [G LOSS: 3.30]
E: 115, [D ACC: %98.44], [D LOSS: 17.14], [G ACC: %100.00], [G LOSS: 4.47]
E: 116, [D ACC: %100.00], [D LOSS: 21.96], [G ACC: %100.00], [G LOSS: 4.47]
E: 117, [D ACC: %98.44], [D LOSS: 32.15], [G ACC: %100.00], [G LOSS: 5.26]
E: 118, [D ACC: %100.00], [D LOSS: 14.38], [G ACC: %100.00], [G LOSS: 4.06]
E: 119, [D ACC: %98.44], [D LOSS: 15.11], [G ACC: %100.00], [G LOSS: 4.27]
E: 120, [D ACC: %98.44], [D LOSS: 34.62], [G ACC: %100.00], [G LOSS: 4.28]
E: 121, [D ACC: %98.44], [D LOSS: 13.04], [G ACC: %100.00], [G LOSS: 4.75]
E: 122, [D ACC: %100.00], [D LOSS: 20.11], [G ACC: %100.00], [G LOSS: 4.07]
E: 123, [D ACC: %98.44], [D LOSS: 10.76], [G ACC: %100.00], [G LOSS: 5.35]
E: 124, [D ACC: %96.88

E: 220, [D ACC: %98.44], [D LOSS: 21.58], [G ACC: %100.00], [G LOSS: 8.05]
E: 221, [D ACC: %100.00], [D LOSS: 8.68], [G ACC: %100.00], [G LOSS: 7.52]
E: 222, [D ACC: %100.00], [D LOSS: 13.58], [G ACC: %100.00], [G LOSS: 7.73]
E: 223, [D ACC: %98.44], [D LOSS: 8.18], [G ACC: %100.00], [G LOSS: 5.67]
E: 224, [D ACC: %98.44], [D LOSS: 31.22], [G ACC: %100.00], [G LOSS: 5.71]
E: 225, [D ACC: %98.44], [D LOSS: 16.14], [G ACC: %100.00], [G LOSS: 4.74]
E: 226, [D ACC: %98.44], [D LOSS: 8.70], [G ACC: %100.00], [G LOSS: 4.74]
E: 227, [D ACC: %100.00], [D LOSS: 19.48], [G ACC: %100.00], [G LOSS: 4.64]
E: 228, [D ACC: %98.44], [D LOSS: 29.91], [G ACC: %100.00], [G LOSS: 5.83]
E: 229, [D ACC: %100.00], [D LOSS: 10.10], [G ACC: %100.00], [G LOSS: 5.62]
E: 230, [D ACC: %100.00], [D LOSS: 13.88], [G ACC: %100.00], [G LOSS: 8.88]
E: 231, [D ACC: %100.00], [D LOSS: 19.75], [G ACC: %100.00], [G LOSS: 10.61]
E: 232, [D ACC: %100.00], [D LOSS: 11.47], [G ACC: %100.00], [G LOSS: 12.61]
E: 233, [D ACC: %98

...

E: 659, [D ACC: %100.00], [D LOSS: 5.47], [G ACC: %100.00], [G LOSS: 0.51]
E: 660, [D ACC: %98.44], [D LOSS: 6.52], [G ACC: %100.00], [G LOSS: 0.68]
E: 661, [D ACC: %98.44], [D LOSS: 6.33], [G ACC: %100.00], [G LOSS: 0.49]
E: 662, [D ACC: %100.00], [D LOSS: 5.42], [G ACC: %100.00], [G LOSS: 0.55]
E: 663, [D ACC: %100.00], [D LOSS: 6.89], [G ACC: %100.00], [G LOSS: 0.52]
E: 664, [D ACC: %98.44], [D LOSS: 5.76], [G ACC: %100.00], [G LOSS: 0.74]
E: 665, [D ACC: %100.00], [D LOSS: 4.76], [G ACC: %100.00], [G LOSS: 0.76]
E: 666, [D ACC: %100.00], [D LOSS: 4.46], [G ACC: %100.00], [G LOSS: 0.91]
E: 667, [D ACC: %100.00], [D LOSS: 4.71], [G ACC: %100.00], [G LOSS: 0.76]
E: 668, [D ACC: %98.44], [D LOSS: 8.79], [G ACC: %100.00], [G LOSS: 0.87]
E: 669, [D ACC: %100.00], [D LOSS: 19.35], [G ACC: %100.00], [G LOSS: 1.51]
E: 670, [D ACC: %100.00], [D LOSS: 6.68], [G ACC: %100.00], [G LOSS: 1.29]
E: 671, [D ACC: %96.88], [D LOSS: 9.78], [G ACC: %100.00], [G LOSS: 2.43]
E: 672, [D ACC: %100.00], [D 

E: 769, [D ACC: %95.31], [D LOSS: 11.60], [G ACC: %62.50], [G LOSS: 101.55]
E: 770, [D ACC: %93.75], [D LOSS: 12.10], [G ACC: %65.62], [G LOSS: 83.69]
E: 771, [D ACC: %90.62], [D LOSS: 36.60], [G ACC: %50.00], [G LOSS: 105.27]
E: 772, [D ACC: %95.31], [D LOSS: 21.98], [G ACC: %56.25], [G LOSS: 123.18]
E: 773, [D ACC: %98.44], [D LOSS: 7.81], [G ACC: %40.62], [G LOSS: 146.17]
E: 774, [D ACC: %95.31], [D LOSS: 10.68], [G ACC: %46.88], [G LOSS: 117.86]
E: 775, [D ACC: %98.44], [D LOSS: 12.91], [G ACC: %62.50], [G LOSS: 71.61]
E: 776, [D ACC: %92.19], [D LOSS: 19.77], [G ACC: %78.12], [G LOSS: 47.49]
E: 777, [D ACC: %92.19], [D LOSS: 17.77], [G ACC: %87.50], [G LOSS: 33.34]
E: 778, [D ACC: %95.31], [D LOSS: 18.85], [G ACC: %93.75], [G LOSS: 29.70]
E: 779, [D ACC: %95.31], [D LOSS: 17.00], [G ACC: %81.25], [G LOSS: 39.31]
E: 780, [D ACC: %100.00], [D LOSS: 5.61], [G ACC: %93.75], [G LOSS: 26.33]
E: 781, [D ACC: %96.88], [D LOSS: 11.66], [G ACC: %87.50], [G LOSS: 25.25]
E: 782, [D ACC: %93.7

E: 879, [D ACC: %100.00], [D LOSS: 11.02], [G ACC: %100.00], [G LOSS: 6.35]
E: 880, [D ACC: %100.00], [D LOSS: 1.59], [G ACC: %100.00], [G LOSS: 7.58]
E: 881, [D ACC: %100.00], [D LOSS: 5.94], [G ACC: %100.00], [G LOSS: 6.60]
E: 882, [D ACC: %98.44], [D LOSS: 4.17], [G ACC: %96.88], [G LOSS: 13.45]
E: 883, [D ACC: %100.00], [D LOSS: 4.11], [G ACC: %93.75], [G LOSS: 36.76]
E: 884, [D ACC: %100.00], [D LOSS: 9.50], [G ACC: %12.50], [G LOSS: 140.94]
E: 885, [D ACC: %100.00], [D LOSS: 2.33], [G ACC: %0.00], [G LOSS: 382.83]
E: 886, [D ACC: %100.00], [D LOSS: 3.43], [G ACC: %3.12], [G LOSS: 284.34]
E: 887, [D ACC: %82.81], [D LOSS: 36.89], [G ACC: %0.00], [G LOSS: 442.73]
E: 888, [D ACC: %100.00], [D LOSS: 3.81], [G ACC: %0.00], [G LOSS: 534.81]
E: 889, [D ACC: %98.44], [D LOSS: 4.27], [G ACC: %0.00], [G LOSS: 496.46]
E: 890, [D ACC: %100.00], [D LOSS: 7.81], [G ACC: %0.00], [G LOSS: 432.62]
E: 891, [D ACC: %100.00], [D LOSS: 3.76], [G ACC: %3.12], [G LOSS: 367.22]
E: 892, [D ACC: %100.00],

E: 989, [D ACC: %100.00], [D LOSS: 4.16], [G ACC: %100.00], [G LOSS: 2.35]
E: 990, [D ACC: %100.00], [D LOSS: 9.11], [G ACC: %96.88], [G LOSS: 13.53]
E: 991, [D ACC: %100.00], [D LOSS: 3.66], [G ACC: %96.88], [G LOSS: 10.13]
E: 992, [D ACC: %100.00], [D LOSS: 4.57], [G ACC: %100.00], [G LOSS: 4.98]
E: 993, [D ACC: %100.00], [D LOSS: 3.45], [G ACC: %96.88], [G LOSS: 7.70]
E: 994, [D ACC: %100.00], [D LOSS: 2.77], [G ACC: %100.00], [G LOSS: 4.90]
E: 995, [D ACC: %100.00], [D LOSS: 4.56], [G ACC: %100.00], [G LOSS: 2.29]
E: 996, [D ACC: %100.00], [D LOSS: 9.84], [G ACC: %100.00], [G LOSS: 2.78]
E: 997, [D ACC: %100.00], [D LOSS: 2.91], [G ACC: %100.00], [G LOSS: 3.69]
E: 998, [D ACC: %100.00], [D LOSS: 2.96], [G ACC: %100.00], [G LOSS: 4.88]
E: 999, [D ACC: %100.00], [D LOSS: 7.11], [G ACC: %100.00], [G LOSS: 7.84]
E: 1000, [D ACC: %100.00], [D LOSS: 5.33], [G ACC: %100.00], [G LOSS: 5.03]
E: 1001, [D ACC: %98.44], [D LOSS: 5.28], [G ACC: %100.00], [G LOSS: 2.50]
E: 1002, [D ACC: %95.31],

E: 1098, [D ACC: %98.44], [D LOSS: 4.04], [G ACC: %100.00], [G LOSS: 4.86]
E: 1099, [D ACC: %100.00], [D LOSS: 2.62], [G ACC: %96.88], [G LOSS: 12.76]
E: 1100, [D ACC: %100.00], [D LOSS: 1.42], [G ACC: %100.00], [G LOSS: 8.07]
E: 1101, [D ACC: %100.00], [D LOSS: 2.85], [G ACC: %96.88], [G LOSS: 14.43]
E: 1102, [D ACC: %100.00], [D LOSS: 1.10], [G ACC: %93.75], [G LOSS: 17.84]
E: 1103, [D ACC: %100.00], [D LOSS: 1.94], [G ACC: %90.62], [G LOSS: 20.49]
E: 1104, [D ACC: %100.00], [D LOSS: 2.55], [G ACC: %100.00], [G LOSS: 6.59]
E: 1105, [D ACC: %100.00], [D LOSS: 5.35], [G ACC: %100.00], [G LOSS: 4.49]
E: 1106, [D ACC: %98.44], [D LOSS: 4.97], [G ACC: %100.00], [G LOSS: 3.19]
E: 1107, [D ACC: %100.00], [D LOSS: 2.94], [G ACC: %100.00], [G LOSS: 3.38]
E: 1108, [D ACC: %100.00], [D LOSS: 1.31], [G ACC: %100.00], [G LOSS: 4.11]
E: 1109, [D ACC: %100.00], [D LOSS: 1.90], [G ACC: %100.00], [G LOSS: 3.21]
E: 1110, [D ACC: %100.00], [D LOSS: 3.58], [G ACC: %100.00], [G LOSS: 2.94]
E: 1111, [D AC

## Plot Loss

In [ ]:
plt.figure(figsize=(20, 18))
plt.plot(train_loss_g, label="Generator Loss");
plt.plot(train_loss_d, label="Discriminator Loss");
plt.legend();

## Plot Accuracy

In [ ]:
plt.figure(figsize=(20, 18))
plt.plot(train_acc_g, label="Generator Accuracy");
plt.plot(train_acc_d, label="Discriminator Accuracy");
plt.legend();

In [ ]:
## save model
pth = os.path.join(models_path, 'gray-gan.h5')
gan.save(pth)

pth = os.path.join(models_path, 'gray-generator.h5')
generator.save(pth)

pth = os.path.join(models_path, 'gray-discriminator.h5')
discriminator.save(pth)